# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import matplotlib as plt
import numpy as np
import geopandas as gpd
import requests
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
del city_data_df["City_ID"]

city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ubatuba,-23.4339,-45.0711,80.53,78,18,5.82,BR,1704653883
1,bredasdorp,-34.5322,20.0403,64.02,66,98,3.49,ZA,1704653883
2,blackmans bay,-43.0167,147.3167,59.13,95,100,1.99,AU,1704653883
3,ust-tsilma,65.4410,52.1498,-0.26,99,90,3.94,RU,1704653883
4,tura,25.5198,90.2201,57.69,54,0,2.68,IN,1704653883
...,...,...,...,...,...,...,...,...,...
582,chibougamau,49.9168,-74.3659,10.02,79,100,6.91,CA,1704653992
583,thinadhoo,0.5333,72.9333,82.94,77,100,14.27,MV,1704653992
584,kyren,51.6828,102.1408,13.95,97,98,1.01,RU,1704653993
585,korenevo,55.6719,38.0078,-7.19,84,0,6.71,RU,1704653993


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
# %%capture --no-display

# Configure the map plot

city_data_df 

gdf = gpd.GeoDataFrame(
    city_data_df, geometry=gpd.points_from_xy(city_data_df['Lng'], city_data_df['Lat'])
)

# Plot the interactive map with points sized based on humidity
map_plot = gdf.hvplot(
    geo=True,
    tiles='CartoLight',
    size='Humidity',
    frame_width=800,
    frame_height=600,
    cmap='Blues',
    hover_cols=['City', 'Humidity']
)

# Show the interactive map
map_plot.opts(title='City Humidity Map')


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values

narrow_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 90) & (city_data_df
['Max Temp'] > 70) & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] == 0), :]

# Drop any rows with null values
narrow_city_data_df = narrow_city_data_df.dropna(how='any')

# Display sample data
narrow_city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,haiku-pauwela,20.9219,-156.3051,75.04,81,0,1.99,US,1704653891
110,gamboru,12.3707,14.2173,73.49,27,0,8.28,NG,1704653902
126,port blair,11.6667,92.7500,79.29,74,0,8.79,IN,1704653904
180,cabo san lucas,22.8909,-109.9124,75.61,57,0,5.26,MX,1704653854
199,jiwani,25.0500,61.7417,73.06,60,0,6.08,PK,1704653916
330,alaghsas,17.0187,8.0168,76.89,12,0,8.05,NE,1704653936
369,wajir,1.7471,40.0573,84.16,54,0,5.75,KE,1704653943
374,bardai,11.6500,7.2333,71.47,14,0,9.08,NG,1704653945
417,kadugli,11.0167,29.7167,74.23,19,0,6.60,SD,1704653957
430,reynosa,26.0833,-98.2833,70.70,67,0,9.22,MX,1704653961


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,haiku-pauwela,20.9219,-156.3051,75.04,81,0,1.99,US,1704653891,
110,gamboru,12.3707,14.2173,73.49,27,0,8.28,NG,1704653902,
126,port blair,11.6667,92.7500,79.29,74,0,8.79,IN,1704653904,
180,cabo san lucas,22.8909,-109.9124,75.61,57,0,5.26,MX,1704653854,
199,jiwani,25.0500,61.7417,73.06,60,0,6.08,PK,1704653916,
330,alaghsas,17.0187,8.0168,76.89,12,0,8.05,NE,1704653936,
369,wajir,1.7471,40.0573,84.16,54,0,5.75,KE,1704653943,
374,bardai,11.6500,7.2333,71.47,14,0,9.08,NG,1704653945,
417,kadugli,11.0167,29.7167,74.23,19,0,6.60,SD,1704653957,
430,reynosa,26.0833,-98.2833,70.70,67,0,9.22,MX,1704653961,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [53]:
# Set parameters to search for a hotel
radius = str(10000)
params = {
    'location': f'[Lat], [Lng]',
    'radius': radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = 'hotel'
    params["bias"] = 'proximity'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
haiku-pauwela - nearest hotel: No hotel found
gamboru - nearest hotel: No hotel found
port blair - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
jiwani - nearest hotel: No hotel found
alaghsas - nearest hotel: No hotel found
wajir - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
kadugli - nearest hotel: No hotel found
reynosa - nearest hotel: No hotel found
fada - nearest hotel: No hotel found
cabedelo - nearest hotel: No hotel found


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,haiku-pauwela,20.9219,-156.3051,75.04,81,0,1.99,US,1704653891,No hotel found
110,gamboru,12.3707,14.2173,73.49,27,0,8.28,NG,1704653902,No hotel found
126,port blair,11.6667,92.7500,79.29,74,0,8.79,IN,1704653904,No hotel found
180,cabo san lucas,22.8909,-109.9124,75.61,57,0,5.26,MX,1704653854,No hotel found
199,jiwani,25.0500,61.7417,73.06,60,0,6.08,PK,1704653916,No hotel found
330,alaghsas,17.0187,8.0168,76.89,12,0,8.05,NE,1704653936,No hotel found
369,wajir,1.7471,40.0573,84.16,54,0,5.75,KE,1704653943,No hotel found
374,bardai,11.6500,7.2333,71.47,14,0,9.08,NG,1704653945,No hotel found
417,kadugli,11.0167,29.7167,74.23,19,0,6.60,SD,1704653957,No hotel found
430,reynosa,26.0833,-98.2833,70.70,67,0,9.22,MX,1704653961,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
%%capture --no-display

# Configure the map plot

# Display the map
# YOUR CODE HERE

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,haiku-pauwela,20.9219,-156.3051,75.04,81,0,1.99,US,1704653891,No hotel found
110,gamboru,12.3707,14.2173,73.49,27,0,8.28,NG,1704653902,No hotel found
126,port blair,11.6667,92.7500,79.29,74,0,8.79,IN,1704653904,No hotel found
180,cabo san lucas,22.8909,-109.9124,75.61,57,0,5.26,MX,1704653854,No hotel found
199,jiwani,25.0500,61.7417,73.06,60,0,6.08,PK,1704653916,No hotel found
330,alaghsas,17.0187,8.0168,76.89,12,0,8.05,NE,1704653936,No hotel found
369,wajir,1.7471,40.0573,84.16,54,0,5.75,KE,1704653943,No hotel found
374,bardai,11.6500,7.2333,71.47,14,0,9.08,NG,1704653945,No hotel found
417,kadugli,11.0167,29.7167,74.23,19,0,6.60,SD,1704653957,No hotel found
430,reynosa,26.0833,-98.2833,70.70,67,0,9.22,MX,1704653961,No hotel found
